# REF Assessment Notebook

### 31st May 2021

### Neil D. Lawrence

This notebook is a helper for reviewing ref papers.


In [ ]:
import ref

In [ ]:
data = ref.assess.data()

In [ ]:
write_data = ref.access.upload()

In [ ]:
INDEX = 207591

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import re

In [ ]:
import pandas as pd
from openpyxl import load_workbook, Workbook
import os

In [ ]:
comment = {}
score = {}

def update_df(df, write_df,
              o_text, o_score,
              s_text, s_score,
              r_text, r_score, interdisciplinary_comment, my_comment, index):
    comment = "{o_text} {o_score} {s_text} {s_score} {r_text} {r_score}".format(o_text=o_text, o_score=o_score, 
                                                                                s_text=s_text, s_score=s_score, 
                                                                                r_text=r_text, r_score=r_score)

    score = o_score + s_score + r_score
    
    # Write index
    print(index)
    write_index = write_df['REF output identifier']=="O" + str(index)
    write_df.at[write_index, 'Comment'] = comment
    write_df.at[write_index, 'Score'] = score
    write_df.at[write_index, 'Comment 2'] = interdisciplinary_comment
    write_df.at[write_index, 'Comment 4'] = my_comment
    filename = os.path.expandvars(os.path.join(ref.access.config['datadirectory'], ref.access.config['upload']))
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    write_df.to_excel(writer,sheet_name=ref.access.config['outputs_sheet'], startrow=3,index=False)
    writer.save()
    

def update_index(df, write_df, index):
    if index not in df.index:
        raise ValueError("Invalid index")
    
    ref.assess.query(df, index)

    o_text = "O: "
    s_text = "S: "
    r_text = "R: "
    o_score = 2
    s_score = 2
    r_score = 2
    
    write_index = write_df[write_df['REF output identifier']=="O" + str(index)].index[0]
    
    print(write_df.at[write_index, 'Comment'])
    if match := re.search('O:\s*(.*)\s*([0-4])\.?\s*S:\s*(.*)\s*([0-4])\.?\s*R:\s*(.*)\s*([0-4])\.?\s*', 
                          write_df.at[write_index, 'Comment']):
        o_text = "O: " + match.group(1).strip()
        s_text = "S: " + match.group(3).strip()
        r_text = "R: " + match.group(5).strip()
        o_score = int(match.group(2))
        s_score = int(match.group(4))
        r_score = int(match.group(6))
    elif len(df['Comment'][index])>0:
        raise ValueError("Could not parse comment section of entry O" + str(index))
    

    o_score_range = widgets.IntSlider(min=0, max=4, step=1, value=o_score)    
    s_score_range = widgets.IntSlider(min=0, max=4, step=1, value=s_score)    
    r_score_range = widgets.IntSlider(min=0, max=4, step=1, value=r_score)    

    
    interdisciplinary_comment = write_df['Comment 2'][write_index]
    my_comment = write_df['Comment 4'][write_index]    

    interact_manual(update_df, o_text=o_text, o_score=o_score_range, 
         s_text=s_text, s_score=s_score_range, 
         r_text=r_text, r_score=r_score_range,
         interdisciplinary_comment=interdisciplinary_comment,
         my_comment=my_comment,
        df=fixed(data), index=fixed(index), write_df=fixed(write_df))


    










In [ ]:
index_select = widgets.Dropdown(options=data.index, value=INDEX)
interact(update_index, index=index_select, df=fixed(data), write_df=fixed(write_data))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 5))
write_data.hist(column='Score', grid=False, bins=np.linspace(-0.5, 12.5, 14), width=0.8, ax=ax)
_ = ax.set_xticks(np.linspace(0,12,13))

In [ ]:
write_data['Score'].count()